In [71]:
import pdb

import requests
import pandas as pd
import re

In [102]:
def has_key(key, resp, pageid):
    return bool(resp['query']["pages"][pageid].get(key))

def get_article_info(title):
    params = {
        "action": "query",
        "format": "json",

        "titles": title,

        "prop": "info|links|categories",
        
        # links
        "pllimit": "max",
        "plnamespace": 0,
        
        # categories
        "cllimit": "max"
    }

    links = []
    categories = []

    def query_info(title, params):

        resp = requests.get(
            url="https://en.wikipedia.org/w/api.php",
            params=params).json()

        pageid = list(resp["query"]['pages'].keys())[0]

        if has_key("links", resp, pageid):
            for link in resp['query']["pages"][pageid]["links"]:
                links.append(link["title"])
        
        if has_key("categories", resp, pageid):
            for cat in resp['query']["pages"][pageid]["categories"]:
                if not bool(re.findall(r"(articles)|(uses)|(commons)", cat["title"], re.I)):
                    categories.append(cat["title"])
        
        if resp.get('continue'):
            params.update(resp.get("continue"))
            query_info(title, params)

    query_info(title, params)
    
    return links, categories

In [108]:
links1, categories1 = get_article_info("Random forest")

links2, categories2 = get_article_info("Decision tree")

In [111]:
matches = 0
for link in links1:
    if link in links2:
        print(link)
        matches += 1
        
matches

Association rule learning
Boosting (machine learning)
CiteSeerX
Decision tree learning
Digital object identifier
International Standard Book Number
PubMed Central
PubMed Identifier


8

In [81]:
string = "Articles containing potentially dated statements from 2019"

bool(re.findall(r"article[s]", string, re.I))

True